# Nanoparticle‐microglial interaction in the ischemic brain is modulated by injury duration and treatment Workflow

##### Citation

Joseph, A., Liao, R., Zhang, M., Helmbrecht, H., Mckenna, M., Filteau, J.R., Nance, E., 2020. Nanoparticle‐microglial interaction in the ischemic brain is modulated by injury duration and treatment. Bioengineering & Translational Medicine.. doi:10.1002/btm2.10175

## Purpose: To split tile scans, pick training and testing image sets, and prepping for running the full VAMPIRE Workflow

The notebook as is was the precise notebook used for the publication cited above.  This state with the data uploaded on our lab drive under this publication title will reproduce the results from our paper when run following protocol on the shared desktop folder location specific in user inputs. 

To use this notebook again, please create a new copy as to not disturb the documentation for this publication. More advanced version of this softward can be found in the NanceLab/fiberf GitHub Repository.

*Step 1: Import necessary packages*

In [44]:
import shutil, os

from glob import glob

import numpy as np
import pandas as pd
from skimage import io
import matplotlib.pyplot as plt
from PIL import Image
from numpy.linalg import inv
import image_slicer
from sklearn.model_selection import train_test_split


%matplotlib inline

Experimental Steps:
1. Move/Download the images for testing and training into a new folder
2. Rename images to insure they include the condition somewhere in them
3. Add a folder for each of your stains into the folder created in step 1
4. Input the name of that folder into 'folder_location' below
5. Input the names of the nuclear stain into 'stain1' and the cell stain into 'stain2' below
6. Insert the number of slices into the np.arange command below
7. Insert the number of slices that you want to split each image into in 'slice number'
8. Increase the random_state_num by one
8. Add a folder labeled 'train' to your desktop
9. Add a folder labeled 'test' to your desktop
10. Within folder 'test' create a folder for each of your conditions

*Step 2: User Inputs*

In [368]:
#file names should be in the current working directory
folder_location = 'Users/hhelmbre/Desktop/cortex' #Input Folder Location here

stain1 = 'dapi'
stain2 = 'iba'
stain3 = 'pi'

#conditions are our four regions
conditions = np.arange(1,16,1)

file_type_init = '.tif'
file_type_new = '.png'

slice_number = 4
#random_state_num for Thal = 13
random_state_num = 3
zoom = '40x'

In [369]:
def folder_cleaner(folder, image_type, zoom):
    k=0
    for files in folder:
        if image_type in str(files) and zoom in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
        
    return folder

*Step 3: Split the Image(s) into Color Channels*

In [370]:
arr = os.listdir(folder_location)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type_init, zoom)

In [371]:
for files in file_list:
    im=io.imread(str(folder_location + '/' + files))
    channel1 = im[:, :, 0]
    channel2 = im[:, :, 1]
    channel3 = im[:, :, 2]
    filename = files.replace(file_type_init, "")
    channel1 = Image.fromarray(np.uint16(channel1))
    channel1.save(str(folder_location + '/' + filename + '_' + stain1 + file_type_new))
    channel2 = Image.fromarray(np.uint16(channel2))
    channel2.save(str(folder_location + '/' + filename + '_' + stain2 + file_type_new))
    channel3 = Image.fromarray(np.uint16(channel3))
    channel3.save(str(folder_location + '/' + filename + '_' + stain3 + file_type_new))

*Step 4: Split the Images*

In [372]:
arr = os.listdir(folder_location)
file_list = np.asarray(arr)
file_list = folder_cleaner(file_list, file_type_new, zoom)

In [373]:
for files in file_list:
    image_slicer.slice(str(folder_location + '/' + files), slice_number)
    os.remove(str(folder_location + '/' + files))

*Step 5: Moving the DAPI and Iba images into their own folders*

In [374]:
arr = os.listdir(folder_location)
file_list1 = np.asarray(arr)
file_list1 = folder_cleaner(file_list1, file_type_new, zoom)

In [375]:
for tiled_images in file_list1:
    conditional = str(str(tiled_images)[-5].isdigit())
    if conditional == 'True':
        if stain1 in tiled_images:
            shutil.move(str(folder_location + '/' + tiled_images), str(folder_location + '/' + stain1 + '/' + tiled_images))
        elif stain2 in tiled_images:
            shutil.move(str(folder_location + '/' + tiled_images), str(folder_location + '/' + stain2 + '/' + tiled_images))
        elif stain3 in tiled_images:
            os.remove(folder_location + '/' + tiled_images)
    else:
        pass

*Step 6: Choose training and testing data sets*

In [376]:
arr = os.listdir(str(folder_location + '/' + stain1))
file_list_train = np.asarray(arr)
file_list_train = folder_cleaner(file_list_train, file_type_new, zoom)

In [377]:
X_train, X_test= train_test_split(file_list_train, test_size=0.20, random_state=random_state_num)

*Step 7: Moving the testing and training DAPI data sets into test and train folders*

In [378]:
for names in file_list_train:
    if names in X_train[:]:
        shutil.move(str(folder_location + '/'+ stain1 + '/' + names), '/Users/hhelmbre/Desktop/train_cor')
    else:
        shutil.move(str(folder_location + '/' + stain1 + '/' + names), '/Users/hhelmbre/Desktop/test_cor')

*Step 8: Renaming the DAPI and Iba datasets according to proper VAMPIRE naming modality*

In [379]:
arr_train1 = os.listdir('/Users/hhelmbre/Desktop/train_cor')
file_list_train1 = np.asarray(arr_train1)
file_list_train1 = folder_cleaner(file_list_train1, file_type_new, zoom)

In [380]:
arr_stain2 = os.listdir(str(folder_location + '/' + stain2))
file_list_stain2 = np.asarray(arr_stain2)
file_list_stain2 = folder_cleaner(file_list_stain2, file_type_new, zoom)

In [381]:
im_number= 1
for names in file_list_train1:
    dapi_name = str(names)

    
    
    if im_number < 10:
        os.rename(str('/Users/hhelmbre/Desktop/train_cor/' + names), str('/Users/hhelmbre/Desktop/train_cor/' + 'xy' + '00' + str(im_number) + 'c2.png'))
        
    elif im_number <100:
        os.rename(str('/Users/hhelmbre/Desktop/train_cor/' + names), str('/Users/hhelmbre/Desktop/train_cor/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str('/Users/hhelmbre/Desktop/train_cor/' + names), str('/Users/hhelmbre/Desktop/train_cor/' + 'xy' + str(im_number) + 'c2.png'))
    
    iba_name = dapi_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(folder_location + '/' + stain2 + '/' + iba_name), str('/Users/hhelmbre/Desktop/train_cor/' + 'xy' + '00' + str(im_number) + 'c1.png'))
        
    elif im_number <100:
        os.rename(str(folder_location + '/' + stain2 + '/' + iba_name), str('/Users/hhelmbre/Desktop/train_cor/' + 'xy' + '0' + str(im_number) + 'c1.png'))
    
    else:
        os.rename(str(folder_location + '/' + stain2 + '/' + iba_name), str('/Users/hhelmbre/Desktop/train_cor/' + 'xy' + str(im_number) + 'c1.png'))
        
    
    im_number +=1

*Step 9: Splitting the test group into the appropriate conditions*

In [382]:
arr_test = os.listdir('/Users/hhelmbre/Desktop/test_cor')
file_list_test = np.asarray(arr_test)
file_list_test = folder_cleaner(file_list_test, file_type_new, zoom)

In [383]:
for slice_numbers in conditions:
    number = str(slice_numbers)
    slices = str('-' + number + '_')

    os.mkdir(str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number))
    os.mkdir(str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number))
    os.mkdir(str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number))
    
    for test_images in file_list_test:
        set1 = str('50')
        set2 = str('56')
        set3 = str('59')
        if set1 in test_images:
            if slices in test_images:
                shutil.move(str('/Users/hhelmbre/Desktop/test_cor/' + test_images), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + test_images))
            else:
                zyx = 1
        if set2 in test_images:
            if slices in test_images:
                shutil.move(str('/Users/hhelmbre/Desktop/test_cor/' + test_images), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + test_images))
            else:
                zyx = 1
        if set3 in test_images:
            if slices in test_images:
                shutil.move(str('/Users/hhelmbre/Desktop/test_cor/' + test_images), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + test_images))
            else:
                zyx = 1

*Step 10: Renaming the test images and getting their appropriate iba stain*



In [384]:
for test_conditions in conditions:
    number = str(test_conditions)
    
    arr_test_condition = os.listdir(str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/'))
    file_list_test_condition = np.asarray(arr_test_condition)
    file_list_test_condition = folder_cleaner(file_list_test_condition, file_type_new, zoom)
    
    im_number = 1
    
    for names in file_list_test_condition:
        dapi_name = str(names)

    
        if im_number < 10:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + 'xy' + '00' + str(im_number) + 'c2.png'))
        
        elif im_number <100:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
        else:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + 'xy' + str(im_number) + 'c2.png'))
    
        iba_name = dapi_name.replace(stain1, stain2)
    
        if im_number < 10:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + 'xy' + '00' + str(im_number) + 'c1.png'))
        
        elif im_number < 100:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
        else:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '56-' + number + '/' + 'xy' + str(im_number) + 'c1.png'))
        
    
        im_number +=1
    

In [385]:
for test_conditions in conditions:
    number = str(test_conditions)
    
    arr_test_condition = os.listdir(str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/'))
    file_list_test_condition = np.asarray(arr_test_condition)
    file_list_test_condition = folder_cleaner(file_list_test_condition, file_type_new, zoom)
    
    im_number = 1
    
    for names in file_list_test_condition:
        dapi_name = str(names)

    
        if im_number < 10:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + 'xy' + '00' + str(im_number) + 'c2.png'))
        
        elif im_number <100:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
        else:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + 'xy' + str(im_number) + 'c2.png'))
    
        iba_name = dapi_name.replace(stain1, stain2)
    
        if im_number < 10:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + 'xy' + '00' + str(im_number) + 'c1.png'))
        
        elif im_number < 100:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
        else:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '50-' + number + '/' + 'xy' + str(im_number) + 'c1.png'))
        
    
        im_number +=1
    

In [386]:
for test_conditions in conditions:
    number = str(test_conditions)
    
    arr_test_condition = os.listdir(str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/'))
    file_list_test_condition = np.asarray(arr_test_condition)
    file_list_test_condition = folder_cleaner(file_list_test_condition, file_type_new, zoom)
    
    im_number = 1
    
    for names in file_list_test_condition:
        dapi_name = str(names)

    
        if im_number < 10:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + 'xy' + '00' + str(im_number) + 'c2.png'))
        
        elif im_number <100:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
        else:
            os.rename(str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/'+ names), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + 'xy' + str(im_number) + 'c2.png'))
    
        iba_name = dapi_name.replace(stain1, stain2)
    
        if im_number < 10:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + 'xy' + '00' + str(im_number) + 'c1.png'))
        
        elif im_number < 100:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
        else:
            os.rename(str(folder_location + '/' + stain2 + '/'+ iba_name), str('/Users/hhelmbre/Desktop/test_cor/' + '59-' + number + '/' + 'xy' + str(im_number) + 'c1.png'))
        
    
        im_number +=1

*Final Experimentalist Steps:*
1. Follow Python 2.7 Protocol on shared Computer for VAMPIRE Analysis
2. Upload all files to the appropriate paper folder in the Shared Google Drive